In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import graphviz

In [2]:
with open('input.txt') as f:
    data = f.read().splitlines()
data

['S â†’ AB', 'A â†’ a', 'B â†’ C | b', 'C â†’ D', 'D â†’ E', 'E â†’ a']

In [4]:
CFG = {}
for i in data:
    print(i)
    var = i[0]
    j = i.split(' | ')
    t = 1
    CFG[var] = []
    CFG[var].append(j[0].split(" → ")[1])
    if len(j) > 1:
        while (t < len(j)):
            CFG[var].append(j[t])
            t += 1
print(CFG)

S â†’ AB


IndexError: list index out of range

# Eliminate Unit Productions

In [5]:
def remove_unit(CFG):
    single_terminals = []
    for i in CFG:
        if len(CFG[i])==1 and CFG[i][0].islower():
            single_terminals.append(i)
    print(single_terminals)
    # Now, checking if any of the single terminals are in RHS of any other state
    while True:
        changed = False
        for i in CFG:
            for j in CFG[i]:
                for l in j:
                    if l in single_terminals:
                        k = single_terminals.index(l)
                        CFG[i].append(j.replace(single_terminals[k], CFG[single_terminals[k]][0]))
                        CFG[i].remove(j)
                        if len(CFG[i])==1:
                            single_terminals.append(i)
                        print(single_terminals)
                        changed = True
        if not changed:
            break
    print(CFG)
    return CFG

In [5]:
CFG = remove_unit(CFG)

[]
{'S': []}


# Elimate Useless Productions

In [6]:
def remove_useless(CFG):
    reachable_states = []
    given_states = []
    terminals = []
    for i in CFG:
        if i not in given_states:
            given_states.append(i)
        for j in CFG[i]:
            for l in j:
                if l not in reachable_states:
                    if l.isupper():
                        reachable_states.append(l)
                    else:
                        terminals.append(l)
    print(reachable_states)
    print(given_states)
    print(terminals)
    unreachable = []
    for i in given_states:
        if i=="S":
            continue
        elif i not in reachable_states:
            unreachable.append(i)
    print(unreachable)
    for i in unreachable:
        CFG.pop(i)
    print(CFG)
    return CFG

In [7]:
CFG = remove_useless(CFG)

['B']
['S', 'A', 'B', 'C', 'D', 'E']
['a', 'a', 'b', 'a', 'a', 'a', 'a']
['A', 'C', 'D', 'E']
{'S': ['aB'], 'B': ['b', 'a']}


# Eliminate Null Productions

In [7]:
def remove_null(CFG):
    # Step 1: Find nullable non-terminals
    nullable = set()
    for nonterm in CFG:
        if '' in CFG[nonterm]:
            nullable.add(nonterm)
    while True:
        new_nullable = set()
        for nonterm, prods in CFG.items():
            for prod in prods:
                if all(s in nullable for s in prod):
                    new_nullable.add(nonterm)
                    break
        if new_nullable == nullable:
            break
        nullable = new_nullable
    
    # Step 2: Construct new productions
    new_prods = {}
    for nonterm, prods in CFG.items():
        new_prods[nonterm] = set()
        for prod in prods:
            new_prod = ['']
            for s in prod:
                if s in nullable:
                    new_prod += [x + s for x in new_prod]
                else:
                    new_prod = [x + s for x in new_prod]
            new_prods[nonterm] |= set(new_prod)
    
    # Step 3: Remove epsilon productions and return the new CFG
    new_CFG = {}
    for nonterm, prods in new_prods.items():
        new_CFG[nonterm] = set()
        for prod in prods:
            if prod != '':
                new_CFG[nonterm].add(prod)
    return new_CFG


In [8]:
#CFG = {'S':['AB'], 'A':['aAA','bB',''], 'B':['bBB','']}
cfg = remove_null(CFG)
for nonterm, prods in cfg.items():
    print(nonterm, '->', ' | '.join(prods))


S -> 


In [9]:
with open('input.txt') as f:
    data = f.read().splitlines()
CFG = {}
for i in data:
    print(i)
    var = i[0]
    j = i.split(' | ')
    t = 1
    CFG[var] = []
    CFG[var].append(j[0].split(" â†’ ")[1])
    if len(j) > 1:
        while (t < len(j)):
            CFG[var].append(j[t])
            t += 1
print(CFG)

S â†’ AB
A â†’ a
B â†’ C | b
C â†’ D
D â†’ E
E â†’ a
{'S': ['AB'], 'A': ['a'], 'B': ['C', 'b'], 'C': ['D'], 'D': ['E'], 'E': ['a']}


In [14]:
cfg1 = remove_unit(CFG)
cfg2 = remove_useless(cfg1)
cfg3 = remove_null(cfg2)
# convert cfg3 from set in dict to list in dict
for nonterm, prods in cfg3.items():
    cfg3[nonterm] = list(prods)
     
print(cfg3)
print("\n\n")
for nonterm, prods in cfg3.items():
    print(nonterm, '->', ' | '.join(prods))

[]
{'S': ['aB'], 'B': ['b', 'a']}
['B']
['S', 'B']
['a', 'b', 'a']
[]
{'S': ['aB'], 'B': ['b', 'a']}
{'S': ['aB'], 'B': ['a', 'b']}



S -> aB
B -> a | b


In [30]:
print(type(cfg3))

<class 'dict'>


In [ ]:
s = 'aB'
# sort s
s = ''.join(sorted(s))

In [20]:
def cfg_to_cnf(cfg):
    # we will start by Eliminate terminals from the RHS of the production if they exist with other non-terminals or terminals
    # In the production rule S0 → aA | Aa, S → aA | Aa, A → aBB and B → Aa, terminal a exists on RHS with non-terminals. So we will replace terminal a with X
    simple = cfg.copy()
    dimple = {}
    print(type(simple))
    variablejar = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']
    # pop used variables for the variablejar
    for i in cfg:
        if i in variablejar:
            variablejar.remove(i)
    # find mixed productions
    mixed = []
    for i in cfg:
        for j in cfg[i]:
            if (len(j)>1):
                s1 = sorted(j)
                if s1[0].isupper() and s1[len(s1)-1].islower():
                    mixed.append(j)
    print(mixed)
    mapped_terminal = {}
   
    for key , prods in simple.items():
            for prod1 in prods:
                # prod = ''.join(sorted(prod1))
                print(prod1)
                if prod1 in mixed:
                    # find terminal symbol and map it to a new variable
                    # add that new mapping to the CFG
                    # replace the terminal symbol with the new variable
                    for i in prod1:
                        if i.islower():
                            print(i, "is lower", prod1)
                            newvar = variablejar.pop(0)
                            if (i not in mapped_terminal):
                                mapped_terminal[i] = newvar
                            else:
                                newvar = mapped_terminal[i]
                                
                            tem_prods = simple[key].copy()
                            tem_prods.remove(prod1)
                            tobeadded  = prod1.replace(i, newvar)
                            print("replace", i, "with", newvar, "in", prod1, "to get", tobeadded)
                            tem_prods.append(tobeadded)
                            if key in dimple:
                                print(dimple[key])
                                dimple[key].append(tobeadded)
                            else:
                                dimple[key] = [tobeadded]
                            if newvar in dimple:
                                dimple[newvar].append(i)
                            dimple[newvar] = [i]
                else: 
                    if key in dimple:
                        dimple[key].append(prod1)
                    else:
                        dimple[key] = [prod1]
                            
    print(dimple)
    return dimple
#CFG = {'S': ['AB'], 'A': ['a'], 'B': ['C', 'b'], 'C': ['D'], 'D': ['E'], 'E': ['a']}
cfg3 = {'S':['bA','aB'], 'A':['bAA','aS','a'], 'B':['aBB','bSSS','b']}
cfg3 = cfg_to_cnf(cfg3)

<class 'dict'>
['bA', 'aB', 'bAA', 'aS', 'aBB', 'bSSS']
bA
b is lower bA
replace b with C in bA to get CA
aB
a is lower aB
replace a with D in aB to get DB
['CA']
bAA
b is lower bAA
replace b with C in bAA to get CAA
aS
a is lower aS
replace a with D in aS to get DS
['CAA']
a
aBB
a is lower aBB
replace a with D in aBB to get DBB
bSSS
b is lower bSSS
replace b with C in bSSS to get CSSS
['DBB']
b
{'S': ['CA', 'DB'], 'C': ['b'], 'D': ['a'], 'A': ['CAA', 'DS', 'a'], 'B': ['DBB', 'CSSS', 'b']}


In [21]:
# function to remove long productions

def generate_new_var(cfg, mfg):
    # generate new variable for first two symbols
    variablejar = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']
    # pop used variables for the variablejar
    for i in cfg:
        if i in variablejar:
            variablejar.remove(i)
    for i in mfg:
        if i in variablejar:
            variablejar.remove(i)
    newvar = variablejar.pop(0)
    return newvar

def remove_long(CFG):
    MFG = {}
    
    for nonterm, prods in CFG.items():
        for prod in prods:
            if len(prod) > 2:
                # generate new variable for first two symbols
                # MFG[generate_new_var(CFG)] = [prod[0:2]]
                newVar = generate_new_var(CFG, MFG)
                print("setting", newVar, "->", prod[0:2])
                
                MFG[newVar] = [prod[0:2]]
                MFG[nonterm] = [newVar+prod[2:]]
                print("reseting (", nonterm, "-> ",prod,") to ",nonterm," -> ", newVar+prod[2:])
            else:
                if nonterm in MFG:
                    MFG[nonterm].append(prod)
                else:
                    MFG[nonterm] = [prod]
                 
                
    print(MFG)
    return MFG


# cfg3 = {'S': ['CA', 'DB'], 'C': ['b'], 'D': ['a'], 'A': ['CAA', 'DS'], 'B': ['DBBBQWE', 'CS']}
cfg4=remove_long(cfg3)
while (not checker(cfg4)):
    cfg4 = remove_long(cfg4)

setting E -> CA
reseting ( A ->  CAA ) to  A  ->  EA
setting F -> DB
reseting ( B ->  DBB ) to  B  ->  FB
setting G -> CS
reseting ( B ->  CSSS ) to  B  ->  GSS
{'S': ['CA', 'DB'], 'C': ['b'], 'D': ['a'], 'E': ['CA'], 'A': ['EA', 'DS', 'a'], 'F': ['DB'], 'B': ['GSS', 'b'], 'G': ['CS']}
setting H -> GS
reseting ( B ->  GSS ) to  B  ->  HS
{'S': ['CA', 'DB'], 'C': ['b'], 'D': ['a'], 'E': ['CA'], 'A': ['EA', 'DS', 'a'], 'F': ['DB'], 'H': ['GS'], 'B': ['HS', 'b'], 'G': ['CS']}


In [19]:
cfg3 = {'S': ['CA', 'DB'], 'C': ['b'], 'D': ['a'], 'A': ['CAA', 'DS'], 'B': ['DBB', 'CS']}

In [23]:
"aA".isupper()

False

In [18]:
def checker(cfg3):
    for i in cfg3:
        for j in cfg3[i]:
            if j.islower() and len(j)==1:
                continue
            elif j.isupper() and len(j)==2:
                continue
            else:
                return False
    return True
            
print(checker(cfg4))

False


# Plot the finite Automata

In [19]:
import graphviz

def plot_automaton(grammar):
    """Plot the finite automaton corresponding to a CFG."""
    # Create a new graph
    graph = graphviz.Digraph(format='png')

    # Add states and transitions for each production in the CFG
    for var, prods in grammar.items():
        # Add the state for the variable
        graph.node(var)
        for prod in prods:
            # If the production is a single terminal symbol, add a transition to a new dummy state
            if len(prod) == 1 and prod[0].islower():
                graph.edge(var, 'D0', label=prod[0])
            else:
                # Add a transition for each symbol in the production
                for i in range(len(prod)):
                    graph.edge(var, prod[i])
                    # If the symbol is a nonterminal that produces a single terminal symbol, add a transition to a dummy state
                    if len(prod[i]) == 1 and prod[i].islower():
                        graph.edge(prod[i], 'D0', label=prod[i])

    # Add a dummy state for single-terminal productions
    graph.node('D0', shape='diamond')

    # Set the graph attributes
    graph.attr(rankdir='LR', size='8,5')

    # Render and display the graph
    graph.render(directory="C:\\Users\\DEVANSH\\Downloads\\THOC_Innovative", view=True)  
    print(graph)

In [20]:
plot_automaton(cfg)

digraph {
	S
	S -> a
	a -> D0 [label=a]
	S -> B
	B
	B -> D0 [label=b]
	B -> D0 [label=a]
	D0 [shape=diamond]
	rankdir=LR size="8,5"
}

